# Constant Bidding Strategy

Date: 14/04/2017

This file implements a constant bidding strategy for real time bidding and outputs validation and test bids.

In [1]:
import pandas as pd
import numpy as np

In [2]:
val = pd.read_csv('data/validation.csv')
test = pd.read_csv('data/test.csv')

In [21]:
# Function for creating columns with strategy bids
def apply_strategy(df, params, test=False):
    col_name = 'strategy_constant_bid'
    col_name_validate = 'strategy_constant_validate'
    if col_name not in df.keys():
        df[col_name] = int(0)
    if col_name_validate not in df.keys():
        df[col_name_validate] = int(0)
        
    df[col_name] = int(params)
     
    if test is False:
        df[col_name_validate] = df[col_name] - df['payprice']
    
    return df

# Function for validating strategy with different parameters
def validate_strategy(df, params):
    col_name = 'strategy_constant_bid'
    col_name_validate = 'strategy_constant_validate'
    
    df_temp = df[df[col_name_validate] > 0]
    
    cash_in_bank = glob_cash_in_bank    
    strategy_impressions = 0
    strategy_clicks = 0
    n_rows_in_budget = 0
    

    for row in df_temp.iterrows():
        row = row[1]
        if cash_in_bank > 0:
            n_rows_in_budget += 1
            if row[col_name] > row['payprice']:
                strategy_impressions += 1
                strategy_clicks += int(row['click'])
                cash_in_bank -= row['payprice']
        else:
            break
    if strategy_impressions>0:        
        strategy_CTR = strategy_clicks*100/strategy_impressions
    else:
        strategy_CTR = 0.0
    if strategy_clicks>0:
        strategy_CPC = (glob_cash_in_bank - cash_in_bank)/strategy_clicks
    else:
        strategy_CPC = 'Inf'

    dfr_index = len(df_results)
    df_results.loc[dfr_index] = ['constant', params, strategy_impressions, strategy_clicks, cash_in_bank, strategy_CTR, strategy_CPC]

In [15]:
glob_cash_in_bank = 6250000 #6250
glob_results = dict()
columns = ['strategy_name', 'strategy_params', 'strategy_impressions', 'strategy_clicks', 'cash_in_bank', 
           'strategy_CTR', 'strategy_CPC']
#del(df_results)
df_results = pd.DataFrame(columns = columns)

In [17]:
# Now we optimize the parameter for constant bidding
param_range = np.arange(0, 300, 20)

for i, c in enumerate(param_range):
    validate_strategy(apply_strategy(val, c),  c)
    print("C: {} | Impressions: {} | Clicks: {} | Cash in Bank: {:.2f}$ | CTR: {:.2f}% | CPC: {:.2f}$".format(c, 
                                                                                                              float(df_results.tail(1).strategy_impressions), 
                                                                                                              float(df_results.tail(1).strategy_clicks), 
                                                                                                              float(df_results.tail(1).cash_in_bank), 
                                                                                                              float(df_results.tail(1).strategy_CTR), 
                                                                                                              float(df_results.tail(1).strategy_CPC)))

C: 0 | Impressions: 0.0 | Clicks: 0.0 | Cash in Bank: 6250000.00$ | CTR: 0.00% | CPC: inf$
C: 20 | Impressions: 32248.0 | Clicks: 13.0 | Cash in Bank: 5901643.00$ | CTR: 0.04% | CPC: 26796.69$
C: 40 | Impressions: 83032.0 | Clicks: 29.0 | Cash in Bank: 4555103.00$ | CTR: 0.03% | CPC: 58444.72$
C: 60 | Impressions: 135288.0 | Clicks: 53.0 | Cash in Bank: 1930755.00$ | CTR: 0.04% | CPC: 81495.19$
C: 80 | Impressions: 145516.0 | Clicks: 77.0 | Cash in Bank: -31.00$ | CTR: 0.05% | CPC: 81169.23$
C: 100 | Impressions: 126535.0 | Clicks: 81.0 | Cash in Bank: -36.00$ | CTR: 0.06% | CPC: 77160.94$
C: 120 | Impressions: 117483.0 | Clicks: 83.0 | Cash in Bank: -25.00$ | CTR: 0.07% | CPC: 75301.51$
C: 140 | Impressions: 110625.0 | Clicks: 82.0 | Cash in Bank: -87.00$ | CTR: 0.07% | CPC: 76220.57$
C: 160 | Impressions: 103404.0 | Clicks: 85.0 | Cash in Bank: -4.00$ | CTR: 0.08% | CPC: 73529.46$
C: 180 | Impressions: 96785.0 | Clicks: 83.0 | Cash in Bank: -46.00$ | CTR: 0.09% | CPC: 75301.76$
C: 20

From the results above, it seems that the bid price of 180 provides the best results, because of the highest CTR and lowest CPC with still a high number of clicks and impressions. 

In [24]:
new_val = apply_strategy(val, 180, test=True)
new_test = apply_strategy(test, 180, test=True)

In [25]:
new_val[['bidid', 'strategy_constant_bid']].to_csv('constant_validation_bids.csv', encoding='utf-8')
new_test[['bidid', 'strategy_constant_bid']].to_csv('constant_testing_bids.csv', encoding='utf-8')